# Inladen data

Lees de data in, voeg het jaartal van de wedstrijd toe en creëer een subset met de wedstrijden vanaf 2010

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

lands = ["Germany", "Scotland", "Hungary", "Switzerland", "Spain", "Croatia", "Italy", "Albania", 
         "England", "Denmark", "Serbia", "Slovenia", "France", "Netherlands", "Austria", "Poland", 
         "Belgium", "Slovakia", "Romania", "Ukraine", "Turkey", "Portugal", "Czech Republic", "Georgia"]

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/international-football-results-from-1872-to-2017/goalscorers.csv
/kaggle/input/international-football-results-from-1872-to-2017/shootouts.csv
/kaggle/input/international-football-results-from-1872-to-2017/results.csv
/kaggle/input/fifaworldranking/fifa_ranking-2024-04-04.csv
/kaggle/input/fifaworldranking/fifa_ranking-2023-07-20.csv


In [ ]:
games = pd.read_csv("/kaggle/input/international-football-results-from-1872-to-2017/results.csv")

games['date'] = pd.to_datetime(games['date'], format='%Y-%m-%d')
games['year'] = games['date'].dt.year
games_since_2010 = games[games['date'] >= "2010-01-01"]

games_since_2010.head()

# Definiëren functies

Maak drie functies voor het ophalen van recente wedstrijdgegevens. Per wedstrijd worden de meest recente wedstrijden van beide teams en de meest recente onderlinge wedstrijden opgehaald en wordt het gemiddelde aantal gescoorde goals en de win en loss rate berekend. Het aantal wedstrijden is variabel en wordt als argument meegegeven bij het aanroepen van de functies.



In [ ]:
def get_stats(row, team):
    if row['home_team'] == team:
        return [row['home_score'], 1 if row['home_score'] > row['away_score'] else 0, 1 if row['home_score'] < row['away_score'] else 0]
    else:
        return [row['away_score'], 1 if row['away_score'] > row['home_score'] else 0, 1 if row['away_score'] < row['home_score'] else 0]
    
def get_recent_stats(row, n_matches, home_away):
    df = games.loc[((games['home_team'] == row[home_away]) | (games['away_team'] == row[home_away]))
                    & (games['date'] < row['date'])].sort_values('date', ascending=False).head(n_matches)
    if not df.empty:
        df[['team_score', 'team_win', 'team_loss']] = df.apply(get_stats, axis=1, args=(row[home_away],), result_type='expand')
    else:
        df[['team_score', 'team_win', 'team_loss']] = pd.DataFrame(columns=['team_score', 'team_win', 'team_loss'])
    return [df['team_score'].mean(), df['team_win'].mean(), df['team_loss'].mean()]

def get_head_to_head_stats(row, n_matches, home_away):
    df = games.loc[((games['home_team'] == row['home_team']) & (games['away_team'] == row['away_team'])
                    | (games['home_team'] == row['away_team']) & (games['away_team'] == row['home_team']))
                   & (games['date'] < row['date'])].sort_values('date', ascending=False).head(n_matches)
    if not df.empty:
        df[['team_score', 'team_win', 'team_loss']] = df.apply(get_stats, axis=1, args=(row[home_away],), result_type='expand')
    else:
        df[['team_score', 'team_win', 'team_loss']] = pd.DataFrame(columns=['team_score', 'team_win', 'team_loss'])
    return [df['team_score'].mean(), df['team_win'].mean(), df['team_loss'].mean()]

# Functies toepassen op dataset

De functies worden toegepast op de dataset met wedstrijden vanaf 2010 om voor elke wedstrijd kolommen toe te voegen met het gemiddelde aantal goals (ag), win rate (wr) en loss rate (lr) voor de recente wedstrijden (rec) en de onderlingen wedstrijden (h2h).

In [ ]:
recent_matches = 5

# test_nl = games_since_2010.loc[(games_since_2010['home_team'] == "Netherlands") | (games_since_2010['away_team'] == "Netherlands")].drop(['tournament', 'city', 'country', 'neutral'], axis=1)
# test_nl = test_nl.head(20)
games_since_2010[['home_rec_ag', 'home_rec_wr', 'home_rec_lr']] = games_since_2010.apply(get_recent_stats, axis=1, args=(recent_matches, 'home_team'), result_type='expand')
games_since_2010[['away_rec_ag', 'away_rec_wr', 'away_rec_lr']] = games_since_2010.apply(get_recent_stats, axis=1, args=(recent_matches, 'away_team'), result_type='expand')
display(games_since_2010.head(10))

In [ ]:
head_to_head_matches = 3

games_since_2010[['home_h2h_ag', 'home_h2h_wr', 'home_h2h_lr']] = games_since_2010.apply(get_head_to_head_stats, axis=1, args=(recent_matches, 'home_team'), result_type='expand')
games_since_2010[['away_h2h_ag', 'away_h2h_wr', 'away_h2h_lr']] = games_since_2010.apply(get_head_to_head_stats, axis=1, args=(recent_matches, 'away_team'), result_type='expand')
display(games_since_2010.head(10))

In [ ]:
games_since_2010.to_csv('games_since_2010.csv', index=False)

# Inladen tweede dataset
Lees de data in, maak de data schoon, bereken ranks en afstanden tussen ranks.

In [2]:
# FIFAranking data inladen en opschonen
fifaranking = pd.read_csv('/kaggle/input/fifaworldranking/fifa_ranking-2024-04-04.csv')
fifaranking['year'] = pd.to_datetime(fifaranking['rank_date']).dt.year
fifaranking = fifaranking[['rank', 'country_full', 'country_abrv', 'total_points', 'confederation', 'year']]
fifaranking['country_full'] = np.where(fifaranking['country_full'] == 'Czechia', "Czech Republic", fifaranking['country_full'])

# calculate mean rank and total_points for each country per year
annual = fifaranking.groupby(['year', 'country_full'])[['rank', 'total_points']].agg('mean')
fifaranking_annual = pd.DataFrame(annual.reset_index())

# make new column calculating the distance between annual ranks for each country
fifaranking_annual['annual_distance_rank'] = fifaranking_annual.groupby('country_full')['rank'].diff().fillna(0).round(1)

# make new column calculating the running minimum (highest rank)
fifaranking_annual = fifaranking_annual.sort_values(by=['country_full', 'year'])
def rolling_minimum(group):
    group['highest_rank_rolling'] = group['rank'].rolling(window=len(group), min_periods=1).min()
    return group
fifaranking_annual = fifaranking_annual.groupby('country_full').apply(rolling_minimum)

# make new column calculating the distance between annual points for each country
fifaranking_annual['annual_distance_points'] = fifaranking_annual['total_points'].diff().fillna(0).round(1)

# rolling average rank per country per year
def rolling_avg(group):
    group['avg_rank_rolling'] = group['rank'].rolling(window=len(group), min_periods=1).mean().round(1)
    return group
fifaranking_annual.reset_index(drop=True, inplace=True)
fifaranking_annual = fifaranking_annual.groupby('country_full').apply(rolling_avg)

#table order
fifaranking_annual = fifaranking_annual[['country_full', 'year', 'rank', 'annual_distance_rank', 'avg_rank_rolling', 'highest_rank_rolling', 'total_points', 'annual_distance_points']]
fifaranking_annual = fifaranking_annual.reset_index(drop=True)

/tmp/ipykernel_1006/3973756966.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  fifaranking_annual = fifaranking_annual.groupby('country_full').apply(rolling_minimum)
/tmp/ipykernel_1006/3973756966.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  fifaranking_annual = fifaranking_annual.groupby('country_full').apply(rolling_avg)


In [3]:
fifaranking_annual

,country_full,year,rank,annual_distance_rank,avg_rank_rolling,highest_rank_rolling,total_points,annual_distance_points
0,Afghanistan,2003,199.000000,0.0,199.0,199.000000,42.833333,0.0
1,Afghanistan,2004,197.500000,-1.5,198.2,197.500000,62.750000,19.9
2,Afghanistan,2005,198.833333,1.3,198.4,197.500000,57.750000,-5.0
3,Afghanistan,2006,181.454545,-17.4,194.2,181.454545,67.363636,9.6
4,Afghanistan,2007,186.250000,4.8,192.6,181.454545,24.333333,-43.0
...,...,...,...,...,...,...,...,...
6667,Zimbabwe,2020,110.250000,-2.4,81.9,46.200000,1180.250000,2.4
6668,Zimbabwe,2021,113.375000,3.1,82.9,46.200000,1160.501250,-19.7
6669,Zimbabwe,2022,123.333333,10.0,84.2,46.200000,1138.560000,-21.9
6670,Zimbabwe,2023,124.714286,1.4,85.5,46.200000,1140.274286,1.7


# Laatste dataset inladen
Dit is dezelfde data als de eerste datatset. Hier worden de punten per jaar berekend. 3 punten voor winnen, 1 voor gelijk spel en 0 voor verliezen. Omdat de data per match opgeschreven is, worden de home teams onder de away teams gezet zodat alle teams in 1 kolom komen te staan. Daarna is het per land gegroepeerd en gesommeerd hoeveel punten ze per jaar hebben gehad. 

In [4]:
## soccer results inladen en opschonen
footballresults = pd.read_csv('/kaggle/input/international-football-results-from-1872-to-2017/results.csv')

## 3,1,0 punten toegeven aan elke wedstrijd
footballresults['punten_home'] = np.where(footballresults['away_score'] < footballresults['home_score'], 3, np.where(footballresults['home_score'] == footballresults['away_score'], 1, 0))
footballresults['punten_away'] = np.where(footballresults['away_score'] > footballresults['home_score'], 3, np.where(footballresults['home_score'] == footballresults['away_score'], 1, 0))

## Home en away teams gegevens scheiden en onderelkaar plakken
home = footballresults[['date', 'home_team', 'punten_home']].rename(columns={'home_team': 'team', 'punten_home': 'punten'})
away = footballresults[['date', 'away_team', 'punten_away']].rename(columns={'away_team': 'team', 'punten_away': 'punten'})
landenpunten = pd.concat([home, away], ignore_index=True)
landenpunten['date'] = pd.to_datetime(landenpunten['date'])
landenpunten['year'] = landenpunten['date'].dt.year
landenpunten = landenpunten.drop(columns=['date'])
landenpunten.dropna(inplace=True)

## Per land het aantal punten per jaar sommeren
landengroepen = pd.DataFrame(landenpunten.groupby(['year', 'team'])['punten'].sum()).reset_index()

## fifa en landen samenvoegen
total_df = fifaranking_annual.merge(landengroepen, left_on=['year', 'country_full'], right_on=['year', 'team'])
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5609 entries, 0 to 5608
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   country_full            5609 non-null   object 
 1   year                    5609 non-null   int32  
 2   rank                    5609 non-null   float64
 3   annual_distance_rank    5609 non-null   float64
 4   avg_rank_rolling        5609 non-null   float64
 5   highest_rank_rolling    5609 non-null   float64
 6   total_points            5609 non-null   float64
 7   annual_distance_points  5609 non-null   float64
 8   team                    5609 non-null   object 
 9   punten                  5609 non-null   int64  
dtypes: float64(6), int32(1), int64(1), object(2)
memory usage: 416.4+ KB


In [5]:
landenpunten

,team,punten,year
0,Scotland,1,1872
1,England,3,1873
2,Scotland,3,1874
3,England,1,1875
4,Scotland,3,1876
...,...,...,...
94734,Venezuela,0,2024
94737,Uruguay,0,2024
94738,Panama,0,2024
94741,Colombia,0,2024


In [6]:
total_df

,country_full,year,rank,annual_distance_rank,avg_rank_rolling,highest_rank_rolling,total_points,annual_distance_points,team,punten
0,Afghanistan,2003,199.000000,0.0,199.0,199.000000,42.833333,0.0,Afghanistan,3
1,Afghanistan,2005,198.833333,1.3,198.4,197.500000,57.750000,-5.0,Afghanistan,3
2,Afghanistan,2006,181.454545,-17.4,194.2,181.454545,67.363636,9.6,Afghanistan,2
3,Afghanistan,2007,186.250000,4.8,192.6,181.454545,24.333333,-43.0,Afghanistan,0
4,Afghanistan,2008,187.000000,0.8,191.7,181.454545,32.750000,8.4,Afghanistan,7
...,...,...,...,...,...,...,...,...,...,...
5604,Zimbabwe,2020,110.250000,-2.4,81.9,46.200000,1180.250000,2.4,Zimbabwe,2
5605,Zimbabwe,2021,113.375000,3.1,82.9,46.200000,1160.501250,-19.7,Zimbabwe,7
5606,Zimbabwe,2022,123.333333,10.0,84.2,46.200000,1138.560000,-21.9,Zimbabwe,4
5607,Zimbabwe,2023,124.714286,1.4,85.5,46.200000,1140.274286,1.7,Zimbabwe,2


In [14]:
total_df[total_df["year"] == 2024]

,country_full,year,rank,annual_distance_rank,avg_rank_rolling,highest_rank_rolling,total_points,annual_distance_points,team,punten
18,Afghanistan,2024,154.5,-2.2,166.5,132.166667,1027.300,6.3,Afghanistan,5
51,Albania,2024,65.0,1.7,78.1,41.666667,1378.895,7.6,Albania,6
84,Algeria,2024,43.0,10.6,51.3,20.500000,1481.475,-31.3,Algeria,15
123,Andorra,2024,164.0,6.7,167.7,132.916667,998.375,-17.0,Andorra,1
156,Angola,2024,93.5,-22.9,89.9,52.583333,1236.935,69.5,Angola,19
...,...,...,...,...,...,...,...,...,...,...
5479,Vietnam,2024,110.0,15.4,117.6,93.750000,1179.560,-57.1,Vietnam,3
5512,Wales,2024,29.0,-1.7,53.9,14.833333,1526.630,7.5,Wales,5
5539,Yemen,2024,153.5,-1.5,140.3,90.800000,1022.495,-0.5,Yemen,2
5575,Zambia,2024,86.5,2.9,60.8,16.900000,1276.985,-10.8,Zambia,3


In [7]:
print(landenpunten[landenpunten['year'] == 2015].value_counts())
print(landengroepen[landengroepen['year'] == 2023].sort_values('punten', ascending=False))

landenNL = landengroepen[landengroepen['team'] == 'Netherlands']
landenNL

team           punten  year
South Korea    3       2015    16
Bhutan         0       2015    11
Maldives       0       2015    11
Japan          3       2015    11
Brazil         3       2015    10
                               ..
Guernsey       0       2015     1
Guadeloupe     0       2015     1
Greenland      1       2015     1
Greece         3       2015     1
Åland Islands  3       2015     1
Name: count, Length: 623, dtype: int64
       year           team  punten
13232  2023         Panama      37
13207  2023         Mexico      36
13302  2023  United States      35
13184  2023         Kuwait      30
13238  2023       Portugal      30
...     ...            ...     ...
13289  2023    Timor-Leste       0
13267  2023        Somalia       0
13100  2023         Brunei       0
13298  2023   Two Sicilies       0
13149  2023      Gibraltar       0

[246 rows x 3 columns]


,year,team,punten
145,1905,Netherlands,6
161,1906,Netherlands,0
175,1907,Netherlands,3
190,1908,Netherlands,12
208,1909,Netherlands,6
...,...,...,...
12604,2020,Netherlands,12
12780,2021,Netherlands,36
12987,2022,Netherlands,31
13217,2023,Netherlands,18


In [8]:
total_df[total_df['year']== 2024]

,country_full,year,rank,annual_distance_rank,avg_rank_rolling,highest_rank_rolling,total_points,annual_distance_points,team,punten
18,Afghanistan,2024,154.5,-2.2,166.5,132.166667,1027.300,6.3,Afghanistan,5
51,Albania,2024,65.0,1.7,78.1,41.666667,1378.895,7.6,Albania,6
84,Algeria,2024,43.0,10.6,51.3,20.500000,1481.475,-31.3,Algeria,15
123,Andorra,2024,164.0,6.7,167.7,132.916667,998.375,-17.0,Andorra,1
156,Angola,2024,93.5,-22.9,89.9,52.583333,1236.935,69.5,Angola,19
...,...,...,...,...,...,...,...,...,...,...
5479,Vietnam,2024,110.0,15.4,117.6,93.750000,1179.560,-57.1,Vietnam,3
5512,Wales,2024,29.0,-1.7,53.9,14.833333,1526.630,7.5,Wales,5
5539,Yemen,2024,153.5,-1.5,140.3,90.800000,1022.495,-0.5,Yemen,2
5575,Zambia,2024,86.5,2.9,60.8,16.900000,1276.985,-10.8,Zambia,3


In [9]:
not_in_both = fifaranking_annual[~fifaranking_annual['country_full'].isin(total_df['team'])]
# not_in_both2 = df[~df['team'].isin(fifaranking_annual['country_full'])]
not_in_both['country_full'].unique()
# not_in_both2['team'].unique()

array(['Brunei Darussalam', 'Cabo Verde', 'Chinese Taipei', 'Congo DR',
       'Curacao', "Côte d'Ivoire", 'IR Iran', 'Korea DPR',
       'Korea Republic', 'Kyrgyz Republic', 'Netherlands Antilles',
       'Sao Tome and Principe', 'Serbia and Montenegro',
       'St Kitts and Nevis', 'St Lucia', 'St Vincent and the Grenadines',
       'The Gambia', 'US Virgin Islands', 'USA', 'Zaire'], dtype=object)

In [10]:
tijdelijk = fifaranking_annual[fifaranking_annual['country_full'].isin(lands)]
tijdelijk["country_full"].unique()

array(['Albania', 'Austria', 'Belgium', 'Croatia', 'Czech Republic',
       'Denmark', 'England', 'France', 'Georgia', 'Germany', 'Hungary',
       'Italy', 'Netherlands', 'Poland', 'Portugal', 'Romania',
       'Scotland', 'Serbia', 'Slovakia', 'Slovenia', 'Spain',
       'Switzerland', 'Turkey', 'Ukraine'], dtype=object)

In [11]:
#Netherlands test
netherlands = fifaranking_annual[fifaranking_annual['country_full'] == 'Netherlands']
netherlands

,country_full,year,rank,annual_distance_rank,avg_rank_rolling,highest_rank_rolling,total_points,annual_distance_points
4181,Netherlands,1992,7.000000,0.0,7.0,7.000000,54.000000,-888.9
4182,Netherlands,1993,8.800000,1.8,7.9,7.000000,55.000000,1.0
4183,Netherlands,1994,6.000000,-2.8,7.3,6.000000,58.100000,3.1
4184,Netherlands,1995,10.000000,4.0,8.0,6.000000,56.400000,-1.7
4185,Netherlands,1996,8.300000,-1.7,8.0,6.000000,58.000000,1.6
4186,Netherlands,1997,9.400000,1.1,8.2,6.000000,59.700000,1.7
4187,Netherlands,1998,12.500000,3.1,8.9,6.000000,60.400000,0.7
4188,Netherlands,1999,12.000000,-0.5,9.2,6.000000,686.500000,626.1
4189,Netherlands,2000,14.333333,2.3,9.8,6.000000,673.166667,-13.3
4190,Netherlands,2001,8.583333,-5.8,9.7,6.000000,711.916667,38.8


In [12]:
#final table
fifaranking_annual

,country_full,year,rank,annual_distance_rank,avg_rank_rolling,highest_rank_rolling,total_points,annual_distance_points
0,Afghanistan,2003,199.000000,0.0,199.0,199.000000,42.833333,0.0
1,Afghanistan,2004,197.500000,-1.5,198.2,197.500000,62.750000,19.9
2,Afghanistan,2005,198.833333,1.3,198.4,197.500000,57.750000,-5.0
3,Afghanistan,2006,181.454545,-17.4,194.2,181.454545,67.363636,9.6
4,Afghanistan,2007,186.250000,4.8,192.6,181.454545,24.333333,-43.0
...,...,...,...,...,...,...,...,...
6667,Zimbabwe,2020,110.250000,-2.4,81.9,46.200000,1180.250000,2.4
6668,Zimbabwe,2021,113.375000,3.1,82.9,46.200000,1160.501250,-19.7
6669,Zimbabwe,2022,123.333333,10.0,84.2,46.200000,1138.560000,-21.9
6670,Zimbabwe,2023,124.714286,1.4,85.5,46.200000,1140.274286,1.7


In [13]:
#ranking in 2014
rank14 = fifaranking_annual[(fifaranking_annual['year'] == '2014') & fifaranking_annual['country_full'].isin(['Spain', 'Netherlands'])]
rank14

,country_full,year,rank,annual_distance_rank,avg_rank_rolling,highest_rank_rolling,total_points,annual_distance_points
